In [ ]:
import os
from os import path

# Third-party
from astropy.constants import G
from astropy.io import fits, ascii
from astropy.stats import median_absolute_deviation
from astropy.table import Table, QTable, join
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
%matplotlib inline
import tqdm
from scipy.stats import beta, binned_statistic

from thejoker import JokerSamples

from twoface.config import TWOFACE_CACHE_PATH
from twoface.samples_analysis import MAP_sample
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, NessRG,
                        StarResult, Status, JokerRun)
from twoface.plot import plot_two_panel, plot_phase_fold
from twoface.mass import get_m2_min, mf, period_at_surface, asini, stellar_radius
from twoface.log import log as logger
logger.setLevel(100)

### TODO: describe Need to download catalogs from `TwoFace`

In [ ]:
plot_path = '../paper/figures/'
# table_path = '../data/'
table_path = '../../twoface/paper/1-catalog/tables/'

In [ ]:
unimodal = QTable.read(path.join(table_path, 'highK-unimodal.fits'), character_as_bytes=False)
clean_uni = unimodal[ (unimodal['clean_flag'] == 0) & (unimodal['converged'])]
high_logg = clean_uni[clean_uni['LOGG'] > 2]
print('{0} unimodal, {1} clean, {2} high logg'
      .format(len(unimodal), len(clean_uni), len(high_logg)))

In [ ]:
# bins = np.linspace(0, 3.1, 15)
# res15 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 15))
# res50 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 50))
# res85 = binned_statistic(np.log10(cat['P'].value), values=cat['e'], bins=bins,
#                          statistic=lambda x: np.percentile(x, 85))

# binc = (res15.bin_edges[:-1]+res15.bin_edges[1:])/2
# plt.plot(binc, res15.statistic)
# plt.plot(binc, res50.statistic)
# plt.plot(binc, res85.statistic)

In [ ]:
def Pcirc(t, M1, R1=None, logg=None):
    if logg is not None:
        R1 = stellar_radius(logg, M1)
        
    rho = M1 / (4/3 * np.pi * R1**3)
    rho_sun = u.Msun / (4/3 * np.pi * u.Rsun**3)
    return (5.9*u.day * (rho/rho_sun)**(-1/2) * (t / (16*u.Gyr))**(3/16)).to(u.day)

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(12, 8.5), sharex=True, sharey=True)

grid = np.arange(2, 3.5+0.1, 0.3)
for i, (l, r) in enumerate(zip(grid, grid+0.5)):
    mask = (clean_uni['LOGG'] > l) & (clean_uni['LOGG'] <= r)
    subcat = clean_uni[mask]
    axes.flat[i].errorbar(subcat['P'].value, subcat['e'], 
                          xerr=subcat['P_err'].value, yerr=subcat['e_err'],
                          marker='o', linestyle='none', alpha=0.8)
    axes.flat[i].axvline(10., zorder=-10, color='#aaaaaa', linestyle='--')
    axes.flat[i].axvline(100., zorder=-10, color='#aaaaaa', linestyle='--')
    axes.flat[i].text(1, 0.95, r'${0:.2f} < \log g \leq {1:.2f}$'.format(l, r), 
                      ha='left', va='top', fontsize=16, backgroundcolor='w')
    
    logg = 0.5*(l + r)
    axes.flat[i].axvline(Pcirc(t=1*u.Gyr, M1=1.36*u.Msun, logg=logg).to(u.day).value, 
                         color='tab:orange')
    
axes.flat[0].set_xlim(0.8, 1800)
axes.flat[0].set_ylim(-0.05, 1.)
axes.flat[0].set_xscale('log')

axes[1, 1].set_xlabel('period, $P$ [day]')
axes[0, 0].set_ylabel('eccentricity, $e$')
axes[1, 0].set_ylabel('eccentricity, $e$')

fig.tight_layout()

fig.savefig(path.join(plot_path, 'P-e-grid.pdf'))

# P vs. e

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10., 4.5), sharex=True)

P_surface = period_at_surface(1.36*u.Msun, high_logg['LOGG'], 
                              high_logg['e'], 0.5*u.Msun)

cs = axes[0].scatter(high_logg['P']/P_surface, high_logg['e'],
                     c=high_logg['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=2, vmax=4, 
                     cmap='viridis')
axes[0].errorbar(high_logg['P']/P_surface, high_logg['e'],
                 yerr=high_logg['e_err'], marker='', linestyle='none',
                 ecolor='#666666', alpha=0.5)

axes[0].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[0].set_xlabel(r'$P / P_{\rm surface}$')
axes[1].set_xlabel(r'$P / P_{\rm surface}$')
axes[0].set_ylabel(r'eccentricity, $e$')

axes[0].set_ylim(-0.05, 0.8)
axes[0].set_xlim(0.7, 1000)
axes[1].set_xscale('log')
axes[0].yaxis.set_ticks(np.arange(0, 0.8+0.2, 0.2))

# ---

cs = axes[1].scatter(high_logg['P']/P_surface, high_logg['K'],
                     c=high_logg['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=2, vmax=4, 
                     cmap='viridis')

axes[1].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[1].set_ylim(1, 250)
axes[1].set_yscale('log')
axes[1].set_ylabel('amplitude, $K$ [{0:latex_inline}]'.format(u.km/u.s))

cax = fig.add_axes([0.87, 0.185, 0.02, 0.76])
cb = fig.colorbar(cs, cax=cax)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log g$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')
cb.set_ticks(np.arange(2, 4+0.1, 0.5))

axes[0].axvspan(8., 12, zorder=-5, color='#aaaaaa', alpha=0.5)

fig.tight_layout()
fig.subplots_adjust(right=0.85, wspace=0.35)

fig.savefig(path.join(plot_path, 'P-e-K.pdf'))

Low P, high e

In [ ]:
def f(pp, e):
    return e > (0.05/1*pp + 0.235)
suh = f(high_logg['P']/P_surface, high_logg['e'])

for row, apid, pp, e, P in zip(high_logg[suh], high_logg[suh]['APOGEE_ID'], 
                               (high_logg['P']/P_surface)[suh], high_logg[suh]['e'], high_logg[suh]['P']):
    print('{0}: P/P_s={1:.2f}, e={2:.2f}, P={3:.10f}'.format(apid, pp, e, P))
    print('\t{0}'.format(row['STARFLAGS']))

In [ ]:
q = np.log10(high_logg['M2_min'] / high_logg['M1'])

fig, axes = plt.subplots(1, 2, figsize=(10., 4.5), sharex=True)

P_surface = period_at_surface(1.36*u.Msun, high_logg['LOGG'], 
                              high_logg['e'], 0.5*u.Msun)

cs = axes[0].scatter(high_logg['P']/P_surface, high_logg['e'],
                     c=q, alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=-1, vmax=0, 
                     cmap='viridis')
axes[0].errorbar(high_logg['P']/P_surface, high_logg['e'],
                 yerr=high_logg['e_err'], marker='', linestyle='none',
                 ecolor='#666666', alpha=0.5)

axes[0].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[0].set_xlabel(r'$P / P_{\rm surface}$')
axes[1].set_xlabel(r'$P / P_{\rm surface}$')
axes[0].set_ylabel(r'eccentricity, $e$')

axes[0].set_ylim(-0.05, 0.8)
axes[0].set_xlim(0.7, 1000)
axes[1].set_xscale('log')
axes[0].yaxis.set_ticks(np.arange(0, 0.8+0.2, 0.2))

# ---

cs = axes[1].scatter(high_logg['P']/P_surface, high_logg['K'],
                     c=q, alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=-1, vmax=0, 
                     cmap='viridis')

axes[1].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[1].set_ylim(1, 250)
axes[1].set_yscale('log')
axes[1].set_ylabel('amplitude, $K$ [{0:latex_inline}]'.format(u.km/u.s))

cax = fig.add_axes([0.87, 0.185, 0.02, 0.76])
cb = fig.colorbar(cs, cax=cax)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log_{10} q$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')
# cb.set_ticks(np.arange(, 4+0.1, 0.5))

fig.tight_layout()
fig.subplots_adjust(right=0.85, wspace=0.35)

# fig.savefig(path.join(plot_path, 'P-e-K.pdf'))

In [ ]:
q = np.log10(np.exp(high_logg['lnAge'].value))

fig, axes = plt.subplots(1, 2, figsize=(10., 4.5), sharex=True)

P_surface = period_at_surface(1.36*u.Msun, high_logg['LOGG'], 
                              high_logg['e'], 0.5*u.Msun)

cs = axes[0].scatter(high_logg['P']/P_surface, high_logg['e'],
                     c=q, alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=-1, vmax=1, 
                     cmap='viridis')

axes[0].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[0].set_xlabel(r'$P / P_{\rm surface}$')
axes[1].set_xlabel(r'$P / P_{\rm surface}$')
axes[0].set_ylabel(r'eccentricity, $e$')

axes[0].set_ylim(-0.05, 0.8)
axes[0].set_xlim(0.7, 1000)
axes[1].set_xscale('log')
axes[0].yaxis.set_ticks(np.arange(0, 0.8+0.2, 0.2))

# ---

cs = axes[1].scatter(high_logg['P']/P_surface, high_logg['K'],
                     c=q, alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=-1, vmax=1, 
                     cmap='viridis')

axes[1].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[1].set_ylim(1, 250)
axes[1].set_yscale('log')
axes[1].set_ylabel('amplitude, $K$ [{0:latex_inline}]'.format(u.km/u.s))

cax = fig.add_axes([0.87, 0.185, 0.02, 0.76])
cb = fig.colorbar(cs, cax=cax)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log_{10} q$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')
# cb.set_ticks(np.arange(, 4+0.1, 0.5))

fig.tight_layout()
fig.subplots_adjust(right=0.85, wspace=0.35)

# a/R vs e

In [ ]:
a = asini(high_logg['P'], high_logg['e'], high_logg['K'], 
          high_logg['M1'], high_logg['M2_min'])

R1 = stellar_radius(high_logg['LOGG'], high_logg['M1'])

In [ ]:
x = (a * (1 - high_logg['e'])) / R1

fig, axes = plt.subplots(1, 2, figsize=(10., 4.5), sharex=True)

cs = axes[0].scatter(x, high_logg['e'],
                     c=high_logg['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=2, vmax=4, 
                     cmap='viridis')
# axes[0].errorbar(x, high_logg['e'],
#                  yerr=high_logg['e_err'], marker='', linestyle='none',
#                  ecolor='#666666', alpha=0.5)

axes[0].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[0].set_xlabel(r'$P / P_{\rm surface}$')
axes[1].set_xlabel(r'$P / P_{\rm surface}$')
axes[0].set_ylabel(r'eccentricity, $e$')

axes[0].set_ylim(-0.05, 1)
axes[0].set_xlim(1E-4, 1E1)
axes[1].set_xscale('log')
axes[0].yaxis.set_ticks(np.arange(0, 1+0.2, 0.2))

# ---

cs = axes[1].scatter(x, high_logg['K'],
                     c=high_logg['LOGG'], alpha=0.75, linewidth=0.5, 
                     edgecolor='#333333', s=25, vmin=2, vmax=4, 
                     cmap='viridis')

axes[1].axhline(1., color='tab:blue', linewidth=1., 
                linestyle='--', zorder=-100, marker='')

axes[1].set_ylim(1, 250)
axes[1].set_yscale('log')
axes[1].set_ylabel('amplitude, $K$ [{0:latex_inline}]'.format(u.km/u.s))

cax = fig.add_axes([0.87, 0.185, 0.02, 0.76])
cb = fig.colorbar(cs, cax=cax)
cb.ax.xaxis.set_ticks_position('top')
cb.ax.xaxis.set_label_position('top')
cb.set_label(r'$\log g$', labelpad=10)
cb.solids.set_rasterized(True) 
cb.solids.set_edgecolor('face')
cb.set_ticks(np.arange(2, 4+0.1, 0.5))

fig.tight_layout()
fig.subplots_adjust(right=0.85, wspace=0.35)

# def f(pp, e):
#     return e > (0.05/1*pp + 0.235)
# suh = f(high_logg['P']/P_surface, high_logg['e'])
# axes[0].scatter( (high_logg['P']/P_surface)[suh], high_logg['e'][suh])

# fig.savefig(path.join(plot_path, 'P-e-K.pdf'))

In [ ]:
age = np.exp(high_logg['lnAge'].value)
age = age[np.isfinite(age) & (age < 12.)]

In [ ]:
plt.hist(age, bins='auto');

In [ ]:
np.mean(age)